# How can we use caching to improve performance of recursive functions?

- Decorators aren't simply wrappers that have no impact on the function they're wrapping
    - They can be used to **actually modify the wrapped function**

- When we cache the returned values from a function, it's called **memoization**
    - **Note**: Using this method sacrifices memory in exchange for faster computation
        - This is because the results need to be stored

- **Recall**: from our previous lesson, we looked at the Fibonnacci sequence
    - The slowest but most elegant solution was the recursive function

In [1]:
def fib(n):
    print(f'Calculating fib({n})')
    if n < 3:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [2]:
fib(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(3)
Calculating

55

- As we can see, all these itermediary calculations were required to get our result
    - Furthermore, a lot of calculations were repeated

### Implementing caching

**Method 1: using a class**

- We'll pre-polutate our cache with the first two Fibonnacci numbers
    - *Why?*
        - This is similar to how we have `if n < 3` in our function
            - Don't use recursion on the first two - they're hard coded

In [3]:
class Fib:
    def __init__(self):
        self.cache = {1:1, 2:1}
        
    def fib(self, n):
        if n not in self.cache:
            print(f'Calculating fib({n})')
            self.cache[n] = self.fib(n-1) + self.fib(n-2)
        return self.cache[n]

- And that's it!
    - Let's try it out

In [4]:
f = Fib()

In [5]:
f.fib(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


55

- Already, way fewer intermediate calculations
    - None of them repeated

- As we've seen in previous videos, **we can use closures instead of classes**
    - i.e. decorators

**Method 2: using closures**

In [6]:
def fib():
    cache = {1:1, 2:1}
    def calc_fib(n):
        if n not in cache:
            print(f'Calculating fib({n})')
            cache[n] = calc_fib(n-1) + calc_fib(n-2)
        return cache[n]
    return calc_fib

In [7]:
f = fib()

In [8]:
f(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


55

In [9]:
f(10)

55

- Re-running it, nothing needs to be recalculated
    - However, if we create another instance of `fib()`, they won't be stored

In [10]:
g = fib()

In [11]:
g(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


55

- Each instance has its own cache
    - Not a global cache

# How can we rewrite these functions to use a decorator?

- We'll write a generic caching decorator
    - Will work regardless of how the numbers are calculated
        - Recursive or not

In [12]:
def memoize_fib(fib):
    cache = {1:1, 2:1}
    def inner(n):
        if n not in cache:
            cache[n] = fib(n)
        return cache[n]
    return inner

In [13]:
@memoize_fib
def fib(n):
    print(f'Calculating fib({n})')
    if n < 3:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [14]:
fib(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


55

- Boom!

# Can we update this to be more general? So it won't require hard-coding the `cache` in the decorator?

- Yes
    - The starting values are in the `fib` function
        - i.e. `fib(1)` won't return an error, it'll return 1
    - Therefore, we can set `cache` to be an **empty dictionary**
    
- We'll also generalize this by swapping out `fn` for `fib`
    - It doesn't really matter in this case
        - However, in the future, if we want to reuse this code, it'll make more sense
        
- Finally, we'll rename the decorator from `memoize_fib` to just `memoize`

In [15]:
def memoize(fn):
    cache = dict()
    def inner(n):
        if n not in cache:
            cache[n] = fn(n)
        return cache[n]
    return inner

In [16]:
@memoize
def fib(n):
    print(f'Calculating fib({n})')
    if n < 3:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [17]:
fib(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


55

- **Note**: as we can see in this run, `Calculating fib(1)` and `Calculating fib(2)` were printed
    - They weren't printed above
        - *Why?*
            - Because, this time, the `cache` was initialized without them

- We can show that this is a truly generic decorator by using it on another recursive function

In [18]:
@memoize
def factorial(n):
    print(f'Calculating factorial of {n}')
    if n < 2:
        return 1
    else:
        return n * factorial(n - 1)

In [19]:
factorial(6)

Calculating factorial of 6
Calculating factorial of 5
Calculating factorial of 4
Calculating factorial of 3
Calculating factorial of 2
Calculating factorial of 1


720

factorial(7

In [20]:
factorial(8)

Calculating factorial of 8
Calculating factorial of 7


40320

# How much faster is this?

- **Recall**: in a previous lecture, when we ran the recursive calculation of the 35th Fibonnacci number, it took ~1.82s to run
    - Let's see how fast this runs

In [21]:
from time import perf_counter

In [22]:
@memoize
def fib(n):
    if n < 3:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [23]:
start = perf_counter()
print(fib(35))
end = perf_counter()
elapsed = end - start
print(elapsed)

9227465
0.00047210000000008634


- Using caching, this took way less time!

# What if we want our memoize to accept any function (not just ones with a single parameter)?

- We can update the `memoize` function to take in `*args` instead of `n`
    - This means the key in `cache` will be a tuple
        - This can lead to issues we'll see in later lectures
- Furthermore, we won't be able to handle `**kwargs`
    - That makes things way more complicated

# At what point does the tradeoff between memory and computation time become a problem with memoization?

- For example, if we called `fib(100000000)`, that would create 100m records in memory
    - This can lead to issues
- **Common solution**: limiting the size of the cache
    - The typical solution drops the least recently used cached values
        - As we add more values to the cache, the older ones are discarded

# Is there a built-in version of this?

- Yes!

In [24]:
from functools import lru_cache

- Here, the lru stands for least recently used

In [27]:
@lru_cache()
def fib(n):
    print(f'Calculating fib({n})')
    if n < 3:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [28]:
fib(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


55

In [29]:
fib(10)

55

In [30]:
fib(11)

Calculating fib(11)


89

- So far, same as before

- The `lru_cache` decorator takes in parameters
    - That's why we used `@lru_cache()` instead of the expected `@lru_cache` above
        - Since we didn't feed in a value, it used the default value (128 items in memory)
        
- Let's try using a smaller cache size

In [36]:
@lru_cache(maxsize=8)
def fib(n):
    print(f'Calculating fib({n})')
    if n < 3:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [37]:
fib(8)

Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


21

In [38]:
fib(9)

Calculating fib(9)


34

In [39]:
fib(9)

34

In [40]:
fib(1)

Calculating fib(1)


1

- As we can see, when we calculated `fib(9)`, the value replaced `fib(1)` in the cache
    - Therefore, when we ran `fib(1)`, it had to be recalculated

- Now, when `fib(1)` was calculated above and added back into the cache, something else had to be deleted
    - *Which value did it replace?*
        - `fib(2)` was the oldest value in the cache
            - Therefore, if we call `fib(2)`, it should be recalculated

In [41]:
fib(2)

Calculating fib(2)


1

- Ayyyy